# Pandas Introduction

Pandas is the de-facto python standar tool to perform data processing and analysis. The current version is v1.1.2 (see http://pandas.pydata.org/). 

You can install it by just running

    conda install pandas
    
In the following we will use both the official pandas documentation and the book "Numerical Python - A practical technique approach for Industry", from Robert Johansson, as the basis for this introduction.

We will also use the seaborn libray to generate plots that are pleasant and clear almost by default. Please install it.

## Starting with pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Series
A series is a convenient container for one-dimensional arrays, which can use descriptive name for indexing and allows for quick access to some descriptive statistics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

datalist = [909976, 8615246, 2872086, 2273305]
print(f"{datalist = }")
s = pd.Series(datalist)
print(s)

The indexes are, by default, of dtype int64. You can access them directly (or by values) as follows

In [ ]:
print(s.index)
print(s.values)

You can change the index to something more descriptive. In this case, those numbers represent the population on a given city, so it would be much more descriptive to use the city name as index:

In [ ]:
s.index = ["Stockholm", "London", "Rome", "Paris"]
s.name = "Population"
print(s)

And you can access to the data directly

In [ ]:
print(s["London"])
print(s.Stockholm)

You can use several indexes to generate a sub-series:

In [ ]:
t = s[["Paris", "Rome"]]
print(t)

You can access basic statistics at a hand:

In [ ]:
s.mean(), s.std(), s.median(), s.quantile(0.5)

In [ ]:
s.describe()

And you can also plot easily, 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
s.plot(ax=axes[0], kind='line', title='line')
s.plot(ax=axes[1], kind='bar', title='bar')
s.plot(ax=axes[2], kind='pie', title='pie')
fig.savefig("fig.pdf", bbox_inches='tight')

## Dataframe
The dataframe is the most important object inside pandas. It allows to represent, access, process, etc multi-dimensional data. 

![Pandas dataframe](https://www.w3resource.com/w3r_images/pandas-data-structure.svg)
Source: https://www.w3resource.com/python-exercises/pandas/index.php

![Pandas dataframe example](https://miro.medium.com/max/1400/1*ZSehcrMtBWN7_qCWq_HiSg.png)
Source: https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

You can initialize a dataframe in several ways. For example, you can use a dictionary or a nested list. For example, you can do something like

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame([[909976, "Sweden"],
                   [8615246, "United Kingdom"],
                   [2872086, "Italy"],
                   [2273305, "France"],
                   [344444, np.nan]])
df

In [ ]:
df.dropna()

In [ ]:
df.fillna("Unknown")

In [ ]:
df = df.dropna()
df

In [ ]:
df.index = ["Stockholm", "London", "Rome", "Paris"]
df.columns = ["Population", "State"]
df

In [ ]:
df.Population

In [ ]:
df.State

You can access a specific row using the `loc` method, which produces a series object (previously you could use `ix` but it is now deprecated. Now use `.loc` or `iloc`)

In [ ]:
df.loc[["London", "Paris"]] # This produces another data frame

In [ ]:
# Or access some rows and columns
df.loc[["London", "Paris"], "Population"] # This produces another data frame, and access a specific column

In [ ]:
df2 = df
df2.index = df.State
df2

## Reading a large csv into a dataframe
Download the population for many european cities from the url: https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits or directly from https://raw.githubusercontent.com/jrjohansson/numerical-python-book-code/master/european_cities.csv
        

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("https://raw.githubusercontent.com/jrjohansson/numerical-python-book-code/master/european_cities.csv")
df.info()

In [ ]:
df.head(7)

In [ ]:
df.tail()

### Processing info
Let's say that you want to have the population as integers. You will need to extract the population , to replace the ',' and to cast the values to integers. You could use the `apply` method to do that 

In [ ]:
df['Numeric Population'] = df.Population.apply(lambda x: int(x.replace(',', ''))) # create a new column
df.head()

In [ ]:
df.dtypes

In [ ]:
print(df["State"].values[:3])

In [ ]:
df["State"] = df["State"].apply(lambda x: x.strip())
df.head()
print(df["State"].values[:3])

You can also change the index,

In [ ]:
df2 = df.set_index("City")
df2 = df2.sort_index()
df2.head()

In [ ]:
df

In [ ]:
df3 = df.set_index(["State", "City"]).sort_index()
df3.head(15)

In [ ]:
df3.loc["Sweden"]

In [ ]:
df3.loc["Sweden"].sort_values("Numeric Population") # Sort by another column

You can count the categorical data:

In [ ]:
df

In [ ]:
city_counts = df.State.value_counts()
city_counts.head(10)

**Exercise: ** How large the total population of all cities within a state? Hint: Use the sum function

In [ ]:
# extract the data indexed by state abd city
dfaux = df[["State", "City", "Numeric Population"]].set_index(["State", "City"])
dfaux = dfaux.groupby(level="State").sum()
dfaux.head(10)

In [ ]:
# Or you can use the groupby method
dfaux = df.groupby("State").sum().sort_values("Numeric Population", ascending=False)
dfaux.head(5)

In [ ]:
dfaux.head(5).drop("Rank", axis=1)

We can also plot

In [ ]:
dfaux

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
city_counts.plot(kind='barh', ax=ax1)
ax1.set_xlabel("# cities in top 105")
dfaux["Numeric Population"].plot(kind='barh', ax=ax2)
ax2.set_xlabel("Total pop. in top 105 cities")

In [ ]:
#print(df.State == "France")
print(df[df.State == "France"])

In [ ]:
# Pivot tables
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
df.pivot_table(values="Numeric Population", index=["State"], aggfunc=np.sum)

### Date series and data frames

In [ ]:
dates = pd.date_range('20130101', periods=6)
print(dates)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

In [ ]:
df.sort_index(axis=1, ascending=False)

In [ ]:
df 

In [ ]:

df["A"] > 0

In [ ]:
df[df.A > 0]

In [ ]:
df[df > 0]

In [ ]:
df[df > 0].fillna(value=5)

In [ ]:
df.to_csv("test.csv")
df.to_excel("test.xlsx")

## Exercises

From : https://www.w3resource.com/python-exercises/pandas/index.php , https://github.com/ajcr/100-pandas-puzzles/blob/master/100-pandas-puzzles.ipynb , https://github.com/guipsamora/pandas_exercises

### Processing a simple dataframe
1. Write a Python program to create and display a DataFrame from a specified dictionary data which has the index labels.

    - `exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew',   'Laura', 'Kevin', 'Jonas']`,
    `'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],`
    `'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],`
    `'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}`
    - `labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']`
    
2. Print the `name` and `score` of the previous dataframe.

3. Plot the socoreper student using matplotlib and seaborn.

4. Show the students with tries larger or equal than 2.

5. Compute the average examination attemps.

6. Save the previous dataframe onto a csv and verify.

7. Save the previous dictionary onto an excel file and verify.

### Solving some exercises

1. Solve the even exercises on https://github.com/ajcr/100-pandas-puzzles/blob/master/100-pandas-puzzles.ipynb 

### Processing covid19 data
Source: https://github.com/CSSEGISandData/COVID-19

## Practice
1) The Colombian government offers public dataset regarding its functioning, at the webpage https://www.datos.gov.co/. As an example, we will get the budget info from Cali, and extract the departments with budget execution smaller than 90%. REF: https://www.datos.gov.co/dataset/Ejecuci-n-Presupuestal-de-Gastos-por-Organismos-a-/qiu9-tp3c
  - Load the data from "http://datos.cali.gov.co/dataset/f7c0d30f-233a-4dbb-bf9e-66d5bf358410/resource/d37f213d-5e5c-4066-bd8f-fc84ca2da0de/download/ejecucion-presupuestal-de-gastos-por-organismos-a-diciembre-31-de-2019.xlsx"
  - (Data cleaning) As you can see, there are several columns that have spaces on their titles. Remove those spaces for easier indexing. Use the method `df.rename` on one column, then devise a way to do it for all columns.
  - Select the departments with budget execution smaller than 90% and print them.
  - For those departments, compute the ratio between the initial budget and any budget addition.
  - Print this info into an excel file

2) Load the gapminder data from "http://bit.ly/2cLzoxH".
  - Plot the gdppercapita as function of the life expectancy. Is there any correlation?


In [ ]:
import pandas as pd
exam_data = pd.DataFrame({'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew',   
                                   'Laura', 'Kevin', 'Jonas'], 
                          'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19], 
                          'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1], 
                          'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']})
exam_data

In [ ]:
labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
exam_data.index = labels
exam_data

In [ ]:
exam_data[["name", "score"]]

In [ ]:
exam_data[exam_data.attempts >= 2]